# Assignment 4 - Document Similarity & Topic Modelling

## Part 1 - Document Similarity

For the first part of this assignment, you will complete the functions `doc_to_synsets` and `similarity_score` which will be used by `document_path_similarity` to find the path similarity between two documents.

The following functions are provided:
* **`convert_tag:`** converts the tag given by `nltk.pos_tag` to a tag used by `wordnet.synsets`. You will need to use this function in `doc_to_synsets`.
* **`document_path_similarity:`** computes the symmetrical path similarity between two documents by finding the synsets in each document using `doc_to_synsets`, then computing similarities using `similarity_score`.

You will need to finish writing the following functions:
* **`doc_to_synsets:`** returns a list of synsets in document. This function should first tokenize and part of speech tag the document using `nltk.word_tokenize` and `nltk.pos_tag`. Then it should find each tokens corresponding synset using `wn.synsets(token, wordnet_tag)`. The first synset match should be used. If there is no match, that token is skipped.
* **`similarity_score:`** returns the normalized similarity score of a list of synsets (s1) onto a second list of synsets (s2). For each synset in s1, find the synset in s2 with the largest similarity value. Sum all of the largest similarity values together and normalize this value by dividing it by the number of largest similarity values found. Be careful with data types, which should be floats. Missing values should be ignored.

Once `doc_to_synsets` and `similarity_score` have been completed, submit to the autograder which will run `test_document_path_similarity` to test that these functions are running correctly. 

*Do not modify the functions `convert_tag`, `document_path_similarity`, and `test_document_path_similarity`.*

In [ ]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger', download_dir = "D:/nltk_data/")

In [54]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd
from sklearn.metrics import accuracy_score

nltk.data.path.append("assets/")

In [10]:
def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None

In [11]:
text = "She dwelt among the untrodden ways"

In [14]:
tokens = nltk.word_tokenize(text)

In [15]:
pos_tagged_tokens = nltk.pos_tag(tokens)

In [33]:
pos_tagged_tokens

[('She', 'PRP'),
 ('dwelt', 'VBD'),
 ('among', 'IN'),
 ('the', 'DT'),
 ('untrodden', 'JJ'),
 ('ways', 'NNS')]

In [19]:
list(map(lambda _tup: convert_tag(_tup[1]) , pos_tagged_tokens))

[None, 'v', None, None, 'a', 'n']

In [24]:
wn.synsets("plhg", "v")

[]

In [34]:
def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    _synsets = []
    
    tokens = nltk.word_tokenize(doc)
    pos_tagged_tokens = nltk.pos_tag(tokens)
    wordnet_tags = list(map(lambda _tuple: convert_tag(_tuple[1]), pos_tagged_tokens))
    
    
    for (token, wn_tag) in zip(tokens, wordnet_tags):
        temp_synsets = wn.synsets(token, wn_tag)
        
        if temp_synsets:
            _synsets.append(temp_synsets[0])
        else:
            continue
    
    return _synsets

In [36]:
doc_to_synsets("Fish are friends.")

[Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]

In [39]:
def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.7333333333333333
    """
    sim_scores = []
    
    for _synset1 in s1:
        
        highest_sscore = {
            "synset": None,
            "score": 0.00
        }
        
        for _synset2 in s2:
            sscore = _synset1.path_similarity(_synset2)
            
            if sscore >= highest_sscore["score"]:
                highest_sscore["synset"] = _synset2
                highest_sscore["score"] = sscore
        
        sim_scores.append(highest_sscore["score"])

    return sum(sim_scores) / len(sim_scores)

In [40]:
similarity_score(doc_to_synsets('I like cats'), doc_to_synsets('I like dogs'))

0.7333333333333334

In [42]:
def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

`paraphrases` is a DataFrame which contains the following columns: `Quality`, `D1`, and `D2`.

`Quality` is an indicator variable which indicates if the two documents `D1` and `D2` are paraphrases of one another (1 for paraphrase, 0 for not paraphrase).

In [43]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('assets/paraphrases.csv')
paraphrases.head()

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...


___

### most_similar_docs

Using `document_path_similarity`, find the pair of documents in paraphrases which has the maximum similarity score.

*This function should return a tuple `(D1, D2, similarity_score)`*

In [44]:
def most_similar_docs():
    
    dpsim_scores = []
    
    for (doc_1, doc_2) in zip(paraphrases.D1, paraphrases.D2):
        dpsim_scores.append(document_path_similarity(doc_1, doc_2))

    offset = np.argmax(dpsim_scores)
    
    return (paraphrases.D1[offset], paraphrases.D2[offset], dpsim_scores[offset])

most_similar_docs()

('"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.',
 '"Iran should be on notice that attempts to remake Iraq in Iran\'s image will be aggressively put down," he said.\n',
 0.9590643274853801)

In [45]:
paraphrases.shape

(20, 3)

### label_accuracy

Provide labels for the twenty pairs of documents by computing the similarity for each pair using `document_path_similarity`. Let the classifier rule be that if the score is greater than 0.75, label is paraphrase (1), else label is paraphrase (0). Report accuracy of the classifier using scikit-learn's accuracy_score.

*This function should return a float.*

In [53]:
def label_accuracy():

    dpsim_scores = []
    
    for (doc_1, doc_2) in zip(paraphrases.D1, paraphrases.D2):
        dpsim_scores.append(document_path_similarity(doc_1, doc_2))
        
    labels = list(map(lambda score: 1 if (score > 0.75) else 0, dpsim_scores))
    return accuracy_score(paraphrases.Quality, labels)
    

label_accuracy()

0.7

## Part 2 - Topic Modelling

For the second part of this assignment, you will use Gensim's LDA (Latent Dirichlet Allocation) model to model topics in `newsgroup_data`. You will first need to finish the code in the cell below by using gensim.models.ldamodel.LdaModel constructor to estimate LDA model parameters on the corpus, and save to the variable `ldamodel`. Extract 10 topics using `corpus` and `id_map`, and with `passes=25` and `random_state=34`.

In [55]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open('assets/newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [56]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

ldamodel = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = id_map, passes = 25, random_state = 34)

### lda_topics

Using `ldamodel`, find a list of the 10 topics and the most significant 10 words in each topic. This should be structured as a list of 10 tuples where each tuple takes on the form:

`(9, '0.068*"space" + 0.036*"nasa" + 0.021*"science" + 0.020*"edu" + 0.019*"data" + 0.017*"shuttle" + 0.015*"launch" + 0.015*"available" + 0.014*"center" + 0.013*"information"')`

for example.

*This function should return a list of tuples.*

In [81]:
def lda_topics():
    return ldamodel.show_topics(num_topics = 10, num_words = 10)

In [100]:
lda_topics();

### topic_distribution

For the new document `new_doc`, find the topic distribution. Remember to use vect.transform on the the new doc, and Sparse2Corpus to convert the sparse matrix to gensim corpus.

*This function should return a list of tuples, where each tuple is `(#topic, probability)`*

In [83]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [85]:
vect.transform(new_doc)

<1x901 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [93]:
corp = gensim.matutils.Sparse2Corpus(vect.transform(new_doc), documents_columns = False)

In [112]:
new = ldamodel[corp]

In [ ]:
def topic_distribution():
    
    # YOUR CODE HERE
    raise NotImplementedError()
    return # Your Answer Here

### topic_names

From the list of the following given topics, assign topic names to the topics you found. If none of these names best matches the topics you found, create a new 1-3 word "title" for the topic.

Topics: Health, Science, Automobiles, Politics, Government, Travel, Computers & IT, Sports, Business, Society & Lifestyle, Religion, Education.

*This function should return a list of 10 strings.*

In [ ]:
def topic_names():
    
    # YOUR CODE HERE
    raise NotImplementedError()
    return # Your Answer Here